In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/goodreads-books-reviews-290312/goodreads_test.csv
/kaggle/input/goodreads-books-reviews-290312/goodreads_sample_submission.csv
/kaggle/input/goodreads-books-reviews-290312/goodreads_train.csv


In [2]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

# Same as before
# checkpoint = "bert-base-uncased"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
# sequences = [
#     "I've been waiting for a HuggingFace course my whole life.",
#     "This course is amazing!",
# ]
# batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

# # This is new
# batch["labels"] = torch.tensor([1, 1])

# optimizer = AdamW(model.parameters())
# loss = model(**batch).loss
# loss.backward()
# optimizer.step()

In [3]:
from datasets import load_dataset, Dataset

# raw_datasets = load_dataset("glue", "mrpc")
# raw_datasets

In [4]:
df = pd.read_csv('/kaggle/input/goodreads-books-reviews-290312/goodreads_train.csv')


In [5]:
df.shape

(900000, 11)

In [6]:
df.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1
1,8842281e1d1347389f2ab93d60773d4d,16981,a5d2c3628987712d0e05c4f90798eb67,3,Recommended by Don Katz. Avail for free in Dec...,Mon Dec 05 10:46:44 -0800 2016,Wed Mar 22 11:37:04 -0700 2017,NaN,NaN,1,0
2,8842281e1d1347389f2ab93d60773d4d,28684704,2ede853b14dc4583f96cf5d120af636f,3,"A fun, fast paced science fiction thriller. I ...",Tue Nov 15 11:29:22 -0800 2016,Mon Mar 20 23:40:27 -0700 2017,Sat Mar 18 23:22:42 -0700 2017,Fri Mar 17 23:45:40 -0700 2017,22,0
3,8842281e1d1347389f2ab93d60773d4d,27161156,ced5675e55cd9d38a524743f5c40996e,0,Recommended reading to understand what is goin...,Wed Nov 09 17:37:04 -0800 2016,Wed Nov 09 17:38:20 -0800 2016,NaN,NaN,5,1
4,8842281e1d1347389f2ab93d60773d4d,25884323,332732725863131279a8e345b63ac33e,4,"I really enjoyed this book, and there is a lot...",Mon Apr 25 09:31:23 -0700 2016,Mon Apr 25 09:31:23 -0700 2016,Sun Jun 26 00:00:00 -0700 2016,Sat May 28 00:00:00 -0700 2016,9,1


In [7]:
df = df.rename(columns={"rating":"label"})

In [8]:
df.head()

,user_id,book_id,review_id,label,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1
1,8842281e1d1347389f2ab93d60773d4d,16981,a5d2c3628987712d0e05c4f90798eb67,3,Recommended by Don Katz. Avail for free in Dec...,Mon Dec 05 10:46:44 -0800 2016,Wed Mar 22 11:37:04 -0700 2017,NaN,NaN,1,0
2,8842281e1d1347389f2ab93d60773d4d,28684704,2ede853b14dc4583f96cf5d120af636f,3,"A fun, fast paced science fiction thriller. I ...",Tue Nov 15 11:29:22 -0800 2016,Mon Mar 20 23:40:27 -0700 2017,Sat Mar 18 23:22:42 -0700 2017,Fri Mar 17 23:45:40 -0700 2017,22,0
3,8842281e1d1347389f2ab93d60773d4d,27161156,ced5675e55cd9d38a524743f5c40996e,0,Recommended reading to understand what is goin...,Wed Nov 09 17:37:04 -0800 2016,Wed Nov 09 17:38:20 -0800 2016,NaN,NaN,5,1
4,8842281e1d1347389f2ab93d60773d4d,25884323,332732725863131279a8e345b63ac33e,4,"I really enjoyed this book, and there is a lot...",Mon Apr 25 09:31:23 -0700 2016,Mon Apr 25 09:31:23 -0700 2016,Sun Jun 26 00:00:00 -0700 2016,Sat May 28 00:00:00 -0700 2016,9,1


In [9]:
# raw_datasets = Dataset.from_pandas(df)

In [10]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

# raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "distilbert-base-uncased"
# checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["review_text"], truncation=True)




Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [11]:
from sklearn.model_selection import train_test_split


In [12]:
df_train, df_test_= train_test_split(df,stratify=df.label, test_size=.2, random_state=10252022)

In [13]:
df_test, df_valid= train_test_split(df_test_,stratify=df_test_.label, test_size=.5, random_state=10252022)

In [14]:
dataset = Dataset.from_pandas(df_train, split='train')
dataset

num_samples = 100
num_classes = 6
train_dataset = dataset.shuffle(seed=42)#.select(range(num_samples * num_classes))
train_dataset

Dataset({
    features: ['user_id', 'book_id', 'review_id', 'label', 'review_text', 'date_added', 'date_updated', 'read_at', 'started_at', 'n_votes', 'n_comments', '__index_level_0__'],
    num_rows: 720000
})

In [15]:
eval_dataset = Dataset.from_pandas(df_test, split='validation')
eval_dataset

Dataset({
    features: ['user_id', 'book_id', 'review_id', 'label', 'review_text', 'date_added', 'date_updated', 'read_at', 'started_at', 'n_votes', 'n_comments', '__index_level_0__'],
    num_rows: 90000
})

In [16]:
tokenized_datasets = train_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/720 [00:00<?, ?ba/s]

In [17]:
eval_tokenized_datasets = eval_dataset.map(tokenize_function, batched=True)


  0%|          | 0/90 [00:00<?, ?ba/s]

In [18]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer", 
                                  num_train_epochs=2.0,
                                  save_steps=7500,
                                  per_device_train_batch_size=16,
                                  report_to="none")

In [19]:
training_args

TrainingArguments(
_n_gpu=2,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
include_inputs_for_metrics=False,
jit_mode_eval=False,
label_names=None,
label_smoothing_facto

In [20]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=6)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_clas

In [21]:
# !pip install wandb

In [22]:
# os.environ["WANDB_DISABLED"] = "true"

In [23]:
# !%env WANDB_SILENT=true
# !%WANDB_SILENT=true


In [24]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=eval_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
#     column_mapping={"review_text": "text", "rating": "label"},
)

In [25]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: review_id, user_id, date_added, read_at, started_at, n_comments, date_updated, __index_level_0__, review_text, book_id, n_votes. If review_id, user_id, date_added, read_at, started_at, n_comments, date_updated, __index_level_0__, review_text, book_id, n_votes are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 720000
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation

Step,Training Loss
500,1.174100
1000,1.022100
1500,0.988900
2000,0.956500
2500,0.940700
3000,0.934300
3500,0.919000
4000,0.922500
4500,0.922500
5000,0.909400


Saving model checkpoint to test-trainer/checkpoint-7500
Configuration saved in test-trainer/checkpoint-7500/config.json
Model weights saved in test-trainer/checkpoint-7500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-7500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-7500/special_tokens_map.json
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to test-trainer/checkpoint-15000
Configuration saved in test-trainer/checkpoint-15000/config.json
Model weights saved in test-trainer/checkpoint-15000/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-15000/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-15000/special_tokens_map.json
/opt/conda/lib

TrainOutput(global_step=45000, training_loss=0.8246343444824219, metrics={'train_runtime': 36274.8499, 'train_samples_per_second': 39.697, 'train_steps_per_second': 1.241, 'total_flos': 1.906368919517756e+17, 'train_loss': 0.8246343444824219, 'epoch': 2.0})

## 

In [26]:
predictions = trainer.predict(eval_tokenized_datasets)
print(predictions.predictions.shape, predictions.label_ids.shape)

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: review_id, user_id, date_added, read_at, started_at, n_comments, date_updated, __index_level_0__, review_text, book_id, n_votes. If review_id, user_id, date_added, read_at, started_at, n_comments, date_updated, __index_level_0__, review_text, book_id, n_votes are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 90000
  Batch size = 16
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


(90000, 6) (90000,)


In [27]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [28]:
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 1.0 MB/s eta 0:00:00


In [29]:
preds

array([5, 4, 5, ..., 5, 1, 0])

In [30]:
predictions.predictions

array([[-1.0794123 , -5.9209127 , -4.2896743 , -0.95853233,  2.2877877 ,
         2.7539039 ],
       [-2.7445545 , -6.622064  , -3.7344275 ,  0.31105074,  2.866371  ,
         2.0237987 ],
       [-1.4300345 , -6.23422   , -5.374443  , -2.3529782 ,  1.4679437 ,
         4.785858  ],
       ...,
       [-1.6587621 , -6.1230793 , -4.0330763 , -0.2159517 ,  2.3315337 ,
         2.5793753 ],
       [ 0.53353924,  1.7891849 , -0.04225715, -1.9744378 , -4.5608134 ,
        -2.3714685 ],
       [ 0.9288011 ,  0.0473108 ,  0.9217966 ,  0.43637633, -2.0147655 ,
        -4.6497993 ]], dtype=float32)

In [31]:
predictions.metrics

{'test_loss': 0.8024478554725647,
 'test_runtime': 901.3727,
 'test_samples_per_second': 99.848,
 'test_steps_per_second': 6.24}

In [32]:
import evaluate

# metric = evaluate.load("glue", "mrpc",average='weighted')
# metric.compute(predictions=preds, references=predictions.label_ids)

In [33]:
# evaluate.load?

In [34]:
from sklearn.metrics import classification_report

In [35]:
print(classification_report(df_test.label, preds))

              precision    recall  f1-score   support

           0       0.65      0.51      0.57      3099
           1       0.61      0.49      0.55      2872
           2       0.58      0.54      0.56      7262
           3       0.64      0.64      0.64     18897
           4       0.63      0.68      0.65     31369
           5       0.74      0.73      0.73     26501

    accuracy                           0.66     90000
   macro avg       0.64      0.60      0.62     90000
weighted avg       0.66      0.66      0.66     90000



In [36]:
df_valid = pd.read_csv('/kaggle/input/goodreads-books-reviews-290312/goodreads_test.csv')
# submission = df_valid[['review_id','rating']]

In [37]:
def create_submission(df_test, save=False):
    dataset = Dataset.from_pandas(df_test)
    tokenized_datasets = dataset.map(tokenize_function, batched=True)
    predictions = trainer.predict(tokenized_datasets)
    preds = np.argmax(predictions.predictions, axis=-1)
    df_test['rating'] = preds
    submission_df = df_test[['review_id','rating']]
    if save:
        submission_df.to_csv('submission.csv', index=False)

    return submission_df

In [38]:
submission_df = create_submission(df_valid)

  0%|          | 0/479 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: review_id, user_id, date_added, read_at, started_at, n_comments, date_updated, review_text, book_id, n_votes. If review_id, user_id, date_added, read_at, started_at, n_comments, date_updated, review_text, book_id, n_votes are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 478033
  Batch size = 16


In [39]:
submission_df.to_csv('submission.csv', index=False)

In [40]:
# chelsy pera: 3033987683